# University of York: APPLIED ARTIFICIAL INTELLIGENCE
## Summative assignment
### **Source file 1**: Data files preprocessing

This is the source file that includes the python script used to preprocess the initial data files.
The resulting combined dataset is called `Final_combined_dataset.csv`.

In [11]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import re
import glob

file_name_col_map = {
    'Child mortality rates_Africa.csv': 'africa',
    'Child mortality rates_Americas.csv' : 'americas',
    'Child mortality rates_Eastern_Mediterranean.csv': 'eastmed',
    'Child mortality rates_South_East_Asia.csv': 'seasia',
    'Child mortality rates_Western_Pacific.csv': 'westpacific',
    'Child mortality rates_Europe.csv': 'europe'
}
df_list = []
for fn, cn in file_name_col_map.items():
    df = pd.read_csv(fn, header=[0,1])
    df['region'] = cn
    df_list.append(df)
df = pd.concat(df_list)
df.columns = [
    col_1 if "Unnamed" in col_0 else f"{col_0}_{col_1}"
    for col_0, col_1 in df.columns
]


In [3]:
def clean_numeric_dataframe(df):
    def clean_value(x):
        if isinstance(x, str):
            return re.sub(r"\[.*\]", "", str(x)).strip()
        return x
    return df.map(clean_value)

# clean numerical columns
df = clean_numeric_dataframe(df)
df.head(10)

,"Countries, territories and areas",Year,Under-five mortality rate (per 1000 live births) (SDG 3.2.1)_ Both sexes,Under-five mortality rate (per 1000 live births) (SDG 3.2.1)_ Male,Under-five mortality rate (per 1000 live births) (SDG 3.2.1)_ Female,Number of deaths among children under-five_ Both sexes,Number of deaths among children under-five_ Male,Number of deaths among children under-five_ Female,region_
0,Algeria,2021,22.34,23.98,20.6,21 567,11 834,9 733,africa
1,Algeria,2020,22.9,24.58,21.14,22 565,12 373,10 192,africa
2,Algeria,2019,23.45,25.13,21.68,23 456,12 847,10 609,africa
3,Algeria,2018,23.98,25.67,22.2,24 191,13 234,10 957,africa
4,Algeria,2017,24.48,26.18,22.68,24 709,13 506,11 203,africa
5,Algeria,2016,24.92,26.64,23.11,24 937,13 624,11 313,africa
6,Algeria,2015,25.3,27.03,23.49,24 898,13 595,11 303,africa
7,Algeria,2014,25.67,27.4,23.85,24 655,13 459,11 196,africa
8,Algeria,2013,26.05,27.79,24.22,24 470,13 355,11 115,africa
9,Algeria,2012,26.49,28.25,24.65,24 174,13 188,10 986,africa


In [4]:
nutrition = pd.read_csv('Infant nutrition data by country.csv')

In [5]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None


def split_year_range(df: pd.DataFrame):
    """
    Finds rows with 'Year' column values representing a range of values,
    delimited by a '-' character. Produces copies of the row for all years 
    in the range. Drops the original row from the dataframe. Returns DataFrame
    containing new rows and list of original indices where splits occurred.
    """
    rows_to_add = []
    indices_to_drop = []
    for i, row in df.iterrows():
        year = row['Year']
        if '-' in year:
            start_year, end_year = year.split('-')
            for y in range(int(start_year), int(end_year) + 1):
                new_row = row.copy()
                new_row['Year'] = str(y)
                rows_to_add.append(new_row)
            indices_to_drop.append(i)
    return pd.DataFrame(rows_to_add).reset_index(drop=True), indices_to_drop

# get new rows from splitting range values
split_df, indices_to_drop = split_year_range(nutrition)
# drop original rows
for x in indices_to_drop:
    nutrition.drop(x, inplace=True)
nutrition.reset_index(inplace=True, drop=True)

# append new rows from split
nutrition = pd.concat([nutrition, split_df]).reset_index(drop=True)

# cast Year to integer
nutrition['Year'] = nutrition["Year"].astype('int')

# sort rows
nutrition = nutrition.sort_values(by=['Countries, territories and areas','Year'], ascending=False)
nutrition.reset_index(inplace=True, drop=True)

# group rows by country and year
grouped = nutrition.groupby(['Countries, territories and areas', 'Year'])
group_list = []
for i, group in grouped:
    # skip groups with only one value
    if group.shape[0] == 1:
        group_list.append(group)
        continue
    # for each group, backfill/frontfill
    group.bfill(inplace=True)
    group.ffill(inplace=True)
    group_list.append(group)
    
# concatenate all groups, reset index, drop copies
nutrition = pd.concat(group_list).reset_index(drop=True).drop_duplicates()

In [6]:
nutrition.head(5)


,"Countries, territories and areas",Year,Early initiation of breastfeeding (%),Infants exclusively breastfed for the first six months of life (%)
0,Afghanistan,2015,40.9,43.1 [40.1-46.2]
2,Afghanistan,2016,40.9,NaN
3,Afghanistan,2018,NaN,57.5
4,Albania,2000,NaN,6.3 [3.0-12.7]
5,Albania,2005,29.9,3.4 [1.1-10.1]


In [7]:
df=pd.merge(df, nutrition, on=['Countries, territories and areas', 'Year'],how='outer')

In [8]:
df.head(5)

,"Countries, territories and areas",Year,Under-five mortality rate (per 1000 live births) (SDG 3.2.1)_ Both sexes,Under-five mortality rate (per 1000 live births) (SDG 3.2.1)_ Male,Under-five mortality rate (per 1000 live births) (SDG 3.2.1)_ Female,Number of deaths among children under-five_ Both sexes,Number of deaths among children under-five_ Male,Number of deaths among children under-five_ Female,region_,Early initiation of breastfeeding (%),Infants exclusively breastfed for the first six months of life (%)
0,Afghanistan,1957,375.91,NaN,NaN,NaN,NaN,NaN,eastmed,NaN,NaN
1,Afghanistan,1958,369.63,NaN,NaN,NaN,NaN,NaN,eastmed,NaN,NaN
2,Afghanistan,1959,363.44,NaN,NaN,NaN,NaN,NaN,eastmed,NaN,NaN
3,Afghanistan,1960,357.3,NaN,NaN,NaN,NaN,NaN,eastmed,NaN,NaN
4,Afghanistan,1961,351.66,NaN,NaN,NaN,NaN,NaN,eastmed,NaN,NaN


In [ ]:
df.to_csv('Final_combined_dataset.csv', index = False)